# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [29]:
def priest(sex, age, respRate, O2Saturation, heartRate, sysBP, temp, alert, inspO2, perfStatus):

  """(str, int, int, float, int, int, float, str, str, str) -> float
  This function determaines the risk score based on the patient parameters like Sex, Age, Respiratory rate in breaths per minute,
  Oxygen saturation as a percent between 0 and 1, Heart rate in beats per minute, Systolic BP in mmHg, Temperature in degrees C,
  Alertness, Inspired Oxygen, Performance Status.
  Risk percentaile is determined based on the scoring algorithm, which can be found at https://www.mdcalc.com/calc/10369/priest-covid-19-clinical-severity-score#evidence.
  After risk percentaile is determined it returns a single numeric value between 0 and 1, but for risk score between 17 and 25, the function returns 0.59 and
  If risk score is greater than or equal to 26, the function returns 0.99 as the risk percentile.
  If any of the inputs are invalid, it will return None, indicating invalid parmeters.

  >>> priest('Male', 55, 15, 0.95, 90, 105, 37.5, 'alert', 'air', 'unrestricted normal activity')
  0.09

  >>> priest('Female', 72, 23, 0.93, 121, 99, 38.6, 'confused or not alert', 'supplemental oxygen', 'limited strenuous activity, can do light activity')
  0.59

  >>> priest('female', 85, 8, 0.94, 100, 98, 37.2, 'alert', 'air', 'unrestricted normal activity')
  0.34
  """
  risk = 0

  if sex.upper() == 'MALE':
    risk += 1
  elif sex.upper() == 'FEMALE':
    risk += 0
  else:
    return None


  if age>=16 and age<= 49:
    risk += 0
  elif age>=50 and age <= 65:
    risk += 2
  elif age>=66 and age<= 80:
    risk += 3
  elif age> 80:
    risk += 4
  else:
    return None


  if respRate<9:
    risk += 3
  elif respRate>=9 and respRate<=11:
    risk += 1
  elif respRate>=12 and respRate<=20:
    risk += 0
  elif respRate>=21 and respRate<=24:
    risk += 2
  else:
    risk += 3


  if O2Saturation> 95/100:
    risk += 0
  elif O2Saturation>= 94/100 and O2Saturation<= 95/100:
    risk += 1
  elif O2Saturation>= 92/100 and O2Saturation<= 93/100:
    risk += 2
  else:
    risk += 3


  if heartRate < 41:
    risk += 3
  elif heartRate>= 41 and heartRate<=50:
    risk += 1
  elif heartRate>=51 and heartRate<= 90:
    risk += 0
  elif heartRate>=91 and heartRate<=110:
    risk += 1
  elif heartRate>=111 and heartRate<=130:
    risk += 2
  else:
    risk += 3


  if sysBP<91:
    risk += 3
  elif sysBP>=91 and sysBP<= 100:
    risk += 2
  elif sysBP>=101 and sysBP<= 110:
    risk += 1
  elif sysBP>=111 and sysBP<=219:
    risk += 0
  else:
    risk += 3


  if temp < 35.1:
    risk += 3
  elif temp >= 35.1 and temp<=36.0:
    risk += 1
  elif temp>= 36.1 and temp<=38.0:
    risk += 0
  elif temp>=38.1 and temp<=39.0:
    risk += 1
  else:
    risk += 2


  if alert.lower() == "alert":
    risk += 0
  elif alert.lower() == "confused or not alert":
    risk += 3
  else:
    return None


  if inspO2.lower() == "air":
    risk += 0
  elif inspO2.lower() == "supplemental oxygen":
    risk += 2
  else:
    return None


  if perfStatus.lower() == "unrestricted normal activity":
    risk += 0
  elif perfStatus.lower() == "limited strenuous activity, can do light activity":
    risk += 1
  elif perfStatus.lower() == "limited activity, can self-care":
    risk += 2
  elif perfStatus.lower() == "limited self-care":
    risk += 3
  elif perfStatus.lower() == "bed/chair bound, no self-care":
    risk += 4
  else:
    return None


  if risk == 0 or risk == 1:
    return 0.01
  elif risk == 2 or risk == 3:
    return 0.02
  elif risk == 4:
    return 0.03
  elif risk == 5:
    return 0.09
  elif risk == 6:
    return 0.15
  elif risk == 7:
    return 0.18
  elif risk == 8:
    return 0.22
  elif risk == 9:
    return 0.26
  elif risk == 10:
    return 0.29
  elif risk == 11:
    return 0.34
  elif risk == 12:
    return 0.38
  elif risk == 13:
    return 0.46
  elif risk == 14:
    return 0.47
  elif risk == 15:
    return 0.49
  elif risk == 16:
    return 0.55
  elif risk>= 17 and risk <= 25:
    return 0.59
  else :
   return 0.99


In [30]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('Male', 55, 15, 0.95, 90, 105, 37.5, 'alert', 'air', 'unrestricted normal activity')
Expecting:
    0.09
ok
Trying:
    priest('Female', 72, 23, 0.93, 121, 99, 38.6, 'confused or not alert', 'supplemental oxygen', 'limited strenuous activity, can do light activity')
Expecting:
    0.59
ok
Trying:
    priest('female', 85, 8, 0.94, 100, 98, 37.2, 'alert', 'air', 'unrestricted normal activity')
Expecting:
    0.34
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [31]:
import requests as req
def find_hospital(age, sex, risk):
  """(int, str, float) -> str
 By retrieving the hospital name, this function determines the appropriate destination for that specific patient based on
 characteristics related to risk percentage, sex, and age (pediatric, adult, or geriatric).
  To obtain the JSON data, this method will use the requests module to make a call to a REST web service.
  then the function reads the JSON data and returns just the hospital's name.
  It will return None if any of the inputs are invalid, indicating that there are invalid parameters, links, or functions.

  >>> find_hospital(49,"FeMaLe",0.65)
  'Emory Dunwoody Medical Center'

  >>> find_hospital(30,"male",0.15)
  'Southwest Hospital and Medical Center'

  >>> find_hospital(60,"male",0.28)
  'Select Specialty Hospital - 'Southwest Hospital and Medical Center'

  >>> find_hospital(35,"female","true")

  >>> find_hospital(26,"male","true")

  """
  source = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={}&sex={}&risk_pct={}".format(age,sex.lower(),risk)
  sourceData = req.get(source)
  if sourceData.status_code == 200:
    hospitalData = sourceData.json()
    if isinstance(hospitalData,dict):
      return hospitalData["hospital"]
    return None

In [32]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(49,"FeMaLe",0.65)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(30,"male",0.15)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(60,"male",0.28)
Expecting:
    'Select Specialty Hospital - 'Southwest Hospital and Medical Center'
**********************************************************************
File "__main__", line 16, in NoName
Failed example:
    find_hospital(60,"male",0.28)
Expected:
    'Select Specialty Hospital - 'Southwest Hospital and Medical Center'
Got:
    'Southwest Hospital and Medical Center'
Trying:
    find_hospital(35,"female","true")
Expecting nothing
ok
Trying:
    find_hospital(26,"male","true")
Expecting nothing
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [33]:
import requests as req
def get_address(hospital_name):
  """(str) -> str
  This function uses the address that is supplied as an input to find the specific hospital.
  Using the requests module, this function will download the file from the following URL: https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw..
  A.json file with hospital data can be found at the provided link. Consequently, the function loads the data from the.json file and looks up the hospital address.
  and gives back the hospital address.

  >>> get_address("UNION GENERAL HOSPITAL")
  '35 HOSPITAL CIRCLE'

  >>> get_address("CRITICAL ACCESS")
  '11740 COLUMBIA STREET'

  >>> get_address("RAYMOND Hospital")

  >>> get_address("MITCHELL COUNTY HOSPITAL")
  '90 STEPHENS STREET'

  >>> get_address("KIMS Hospital")

  """
  hospital = hospital_name.upper()
  dataSource = req.get("https://hds5210-data.s3.amazonaws.com/ga_hospitals.json")
  if dataSource.status_code == 200:
    patientRecords = dataSource.json()
    hospitals = list(patientRecords.keys())
    if hospital in hospitals:
      return patientRecords[hospital]['ADDRESS']
    return None
    return None

## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [34]:
import requests as req
import csv
def process_people(source):
  """(str) -> dict
  This function returns the entire patient details in the dictonary format, by using the file location as its only parameter.
  The file location has a people.psv file. It is a pipe-delimited (|) file with columns that match the inputs for the PRIEST calculation above.
  The function will use all the previous functions above to process all of these rows, and determine the risk, hospital name and address,
  and return a list whose items are a dictionary like this: { patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, risk, hospitalName, hospitalAddress]}.
  """
  sourceData = req.get(source)

  if sourceData.status_code == 200:
    csv = [i.split('|') for i in sourceData.text.split('\n')]
    if(len(csv[-1][0])==0):
      csv.pop(-1)
    csv.pop(0)
    patientRecordsDict = {}
    for row in csv:
      key = row.pop(0)
      patientRecordsList = [row[0],int(row[1]),float(row[2]),float(row[3]),float(row[4]),float(row[5]),float(row[6]),row[7],row[8],row[9]]
      patientRecordsList.append(priest(patientRecordsList[0],patientRecordsList[1],patientRecordsList[2],patientRecordsList[3],patientRecordsList[4],patientRecordsList[5],patientRecordsList[6],patientRecordsList[7],patientRecordsList[8],patientRecordsList[9]))
      patientRecordsList.append(find_hospital(patientRecordsList[1],patientRecordsList[0],patientRecordsList[-1]))
      patientRecordsList.append(get_address(patientRecordsList[-1].upper()))
      patientRecordsDict[key]= patientRecordsList

    return patientRecordsDict
    return None

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [35]:
import requests as req

resultData = req.get('https://hds5210-data.s3.amazonaws.com/people_results.json')

obtainedData = process_people('https://hds5210-data.s3.amazonaws.com/people.psv')

resultData.json() == obtainedData

True

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---